In [1]:
import os
import sys
import logging

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from PIL import Image
import numpy as np
np.random.seed(0)
import cv2

import torch
from torchvision import transforms

from utils.arguments import load_opt_command, load_config_dict_to_opt, load_opt_from_config_files


from xdecoder.BaseModel import BaseModel
from xdecoder import build_model
from utils.visualizer import Visualizer
from utils.distributed import init_distributed


logger = logging.getLogger(__name__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Invalid MIT-MAGIC-COOKIE-1 key

In [2]:
def main(args=None):
    '''
    Main execution point for PyLearn.
    '''
    opt, cmdline_args = load_opt_command(args)
    if cmdline_args.user_dir:
        absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
        opt['user_dir'] = absolute_user_dir
    opt = init_distributed(opt)

    # META DATA
    pretrained_pth = os.path.join(opt['WEIGHT'])
    if 'novg' not in pretrained_pth:
        assert False, "Using the ckpt without visual genome training data will be much better."
    output_root = './output'
    image_pth = 'images/london.jpg'

    model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()
    model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(["background"], is_eval=False)

    t = []
    t.append(transforms.Resize(224, interpolation=Image.BICUBIC))
    transform = transforms.Compose(t)

    with torch.no_grad():
        image_ori = Image.open(image_pth).convert("RGB")
        width = image_ori.size[0]
        height = image_ori.size[1]
        image = transform(image_ori)
        image = np.asarray(image)
        image_ori = np.asarray(image_ori)
        images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()

        batch_inputs = [{'image': images, 'height': height, 'width': width, 'image_id': 0}]
        outputs = model.model.evaluate_captioning(batch_inputs)
        text = outputs[-1]['captioning_text']

        image_ori = image_ori[:,:,::-1].copy()
        cv2.rectangle(image_ori, (0, 0), (width, 60), (0,0,0), -1)
        font                   = cv2.FONT_HERSHEY_DUPLEX
        fontScale              = 1.2
        thickness              = 2
        lineType               = 2
        bottomLeftCornerOfText = (10, 40)
        fontColor              = [255,255,255]
        cv2.putText(image_ori, text,
            bottomLeftCornerOfText,
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType)

        if not os.path.exists(output_root):
            os.makedirs(output_root)
        cv2.imwrite(os.path.join(output_root, 'captioning.png'), image_ori)

In [3]:
main()

usage: ipykernel_launcher.py [-h] --conf_files CONF_FILES [CONF_FILES ...]
                             [--user_dir USER_DIR]
                             [--config_overrides [CONFIG_OVERRIDES ...]]
                             [--overrides ...]
                             command
ipykernel_launcher.py: error: the following arguments are required: command, --conf_files


SystemExit: 2

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
